In [412]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [413]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [414]:
conn = sq.connect('../Data-Pipelines/Outputs/CensusBureau.db')
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#ACS52021
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021_1]', conn)
ACS52021_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021_2]', conn)
ACS52021_2 = pd.DataFrame(sql_query)
ACS52021 = ACS52021_1.merge(ACS52021_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52021.drop(columns = 'Source', inplace = True)
dfs2021 = [ACS52021, PL2020]
data = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)

In [415]:
#under 5, 5 to 9, 10 to 14, total
cols = ['NAME', 'Population', 'Age:Under 5'#, 'Age:5 to 9', 'Age:10 to 14'
       ]
data = data[cols]
data.head()


,NAME,Population,Age:Under 5
0,"Anderson County, Tennessee",77123.0,4035.0
1,"Bedford County, Tennessee",50237.0,3347.0
2,"Benton County, Tennessee",15864.0,756.0
3,"Bledsoe County, Tennessee",14913.0,532.0
4,"Blount County, Tennessee",135280.0,6597.0


In [416]:
geos = ['Cannon County', 'Cheatham County', 'Davidson County', 'Dickson County', 'Macon County', 'Maury County', 
        'Robertson County', 'Rutherford County', 'Smith County', 'Sumner County', 'Trousdale County', 'Williamson County', 
        'Wilson County', 'Montgomery County', 'Stewart County', 'Houston County', 'Humphreys County'
       ]
data = data.set_index('NAME').transpose()
data = data.rename(columns = namestocommon)
data = data.rename(columns = {'Cannon County, Tennessee': 'Cannon County'})
data = data[geos]


In [417]:
data.head()

NAME,Cannon County,Cheatham County,Davidson County,Dickson County,Macon County,Maury County,Robertson County,Rutherford County,Smith County,Sumner County,Trousdale County,Williamson County,Wilson County,Montgomery County,Stewart County,Houston County,Humphreys County
Population,14506.0,41072.0,715884.0,54315.0,25216.0,100974.0,72803.0,341486.0,19904.0,196281.0,11615.0,247726.0,147737.0,220069.0,13657.0,8283.0,18990.0
Age:Under 5,789.0,2250.0,46175.0,3168.0,1736.0,6532.0,4456.0,21410.0,1120.0,11438.0,720.0,13877.0,8587.0,17649.0,807.0,395.0,968.0


In [418]:
thelist = [data['Cannon County'], data['Cheatham County'], data['Davidson County'], data['Dickson County'], data['Macon County'], data['Maury County'], 
        data['Robertson County'], data['Rutherford County'], data['Smith County'], data['Sumner County'], data['Trousdale County'], data['Williamson County'], 
        data['Wilson County'], data['Montgomery County'], data['Stewart County'], data['Houston County'], data['Humphreys County']
       ]
data['Total Region'] = sum(thelist)
thelist = [data['Cannon County'], data['Cheatham County'], data['Davidson County'], data['Dickson County'], data['Macon County'], data['Maury County'], 
        data['Robertson County'], data['Rutherford County'], data['Smith County'], data['Sumner County'], data['Trousdale County'], data['Williamson County'], 
        data['Wilson County']
       ]
data['Nashville MSA'] = sum(thelist)

In [419]:
data = data.transpose().reset_index()
data.head()

,NAME,Population,Age:Under 5
0,Cannon County,14506.0,789.0
1,Cheatham County,41072.0,2250.0
2,Davidson County,715884.0,46175.0
3,Dickson County,54315.0,3168.0
4,Macon County,25216.0,1736.0


In [420]:
data = data.rename(columns = {'Age:Under 5': 'Children Under 5'})

In [421]:
data['Percent of Population Children Under 5'] = percent(data['Children Under 5'], data['Population'])

In [422]:
data

,NAME,Population,Children Under 5,Percent of Population Children Under 5
0,Cannon County,14506.0,789.0,5.439129
1,Cheatham County,41072.0,2250.0,5.478185
2,Davidson County,715884.0,46175.0,6.450067
3,Dickson County,54315.0,3168.0,5.832643
4,Macon County,25216.0,1736.0,6.884518
5,Maury County,100974.0,6532.0,6.468992
6,Robertson County,72803.0,4456.0,6.120627
7,Rutherford County,341486.0,21410.0,6.269657
8,Smith County,19904.0,1120.0,5.627010
9,Sumner County,196281.0,11438.0,5.827360


In [423]:
original = data

In [424]:
childcare = pd.read_csv('childcare1.csv')

In [425]:
childcare.head()

,County,Licensed Capacity
0,Davidson,16.0
1,Davidson,106.0
2,Davidson,164.0
3,Williamson,NaN
4,Montgomery,64.0


In [426]:
childcare['NAME'] = childcare['County'] + ' County'
childcare = childcare.drop(columns = 'County')

In [427]:
childcare['Licensed Capacity'].fillna(0, inplace = True)

In [428]:
childcare['Licensed Capacity'] = childcare['Licensed Capacity'].astype(float)

In [429]:
grouped = childcare.groupby('NAME').sum()

In [430]:
grouped = grouped.reset_index(drop = False)

In [431]:
grouped

,NAME,Licensed Capacity
0,Cannon County,164.0
1,Cheatham County,1881.0
2,Davidson County,28570.0
3,Dickson County,1328.0
4,Houston County,155.0
5,Humphreys County,379.0
6,Macon County,498.0
7,Maury County,2724.0
8,Montgomery County,8738.0
9,Robertson County,2237.0


In [432]:
df = grouped.set_index('NAME').transpose()
data = df
thelist = [data['Cannon County'], data['Cheatham County'], data['Davidson County'], data['Dickson County'], data['Macon County'], data['Maury County'], 
        data['Robertson County'], data['Rutherford County'], data['Smith County'], data['Sumner County'], data['Trousdale County'], data['Williamson County'], 
        data['Wilson County'], data['Montgomery County'], data['Stewart County'], data['Houston County'], data['Humphreys County']
       ]
data['Total Region'] = sum(thelist)
thelist = [data['Cannon County'], data['Cheatham County'], data['Davidson County'], data['Dickson County'], data['Macon County'], data['Maury County'], 
        data['Robertson County'], data['Rutherford County'], data['Smith County'], data['Sumner County'], data['Trousdale County'], data['Williamson County'], 
        data['Wilson County']
       ]
data['Nashville MSA'] = sum(thelist)
grouped = data

In [433]:
grouped = grouped.transpose().reset_index(drop = False)

In [434]:
grouped.head()

,NAME,Licensed Capacity
0,Cannon County,164.0
1,Cheatham County,1881.0
2,Davidson County,28570.0
3,Dickson County,1328.0
4,Houston County,155.0


In [435]:
df = original.merge(grouped, on = 'NAME')

In [436]:
df

,NAME,Population,Children Under 5,Percent of Population Children Under 5,Licensed Capacity
0,Cannon County,14506.0,789.0,5.439129,164.0
1,Cheatham County,41072.0,2250.0,5.478185,1881.0
2,Davidson County,715884.0,46175.0,6.450067,28570.0
3,Dickson County,54315.0,3168.0,5.832643,1328.0
4,Macon County,25216.0,1736.0,6.884518,498.0
5,Maury County,100974.0,6532.0,6.468992,2724.0
6,Robertson County,72803.0,4456.0,6.120627,2237.0
7,Rutherford County,341486.0,21410.0,6.269657,12710.0
8,Smith County,19904.0,1120.0,5.627010,380.0
9,Sumner County,196281.0,11438.0,5.827360,6453.0


In [437]:
df['Difference'] = df['Licensed Capacity'] - df['Children Under 5']
df['Percent of Children Under 5 with a Spot'] = percent(df['Licensed Capacity'], df['Children Under 5'])

In [438]:
df

,NAME,Population,Children Under 5,Percent of Population Children Under 5,Licensed Capacity,Difference,Percent of Children Under 5 with a Spot
0,Cannon County,14506.0,789.0,5.439129,164.0,-625.0,20.785805
1,Cheatham County,41072.0,2250.0,5.478185,1881.0,-369.0,83.600000
2,Davidson County,715884.0,46175.0,6.450067,28570.0,-17605.0,61.873308
3,Dickson County,54315.0,3168.0,5.832643,1328.0,-1840.0,41.919192
4,Macon County,25216.0,1736.0,6.884518,498.0,-1238.0,28.686636
5,Maury County,100974.0,6532.0,6.468992,2724.0,-3808.0,41.702388
6,Robertson County,72803.0,4456.0,6.120627,2237.0,-2219.0,50.201975
7,Rutherford County,341486.0,21410.0,6.269657,12710.0,-8700.0,59.364783
8,Smith County,19904.0,1120.0,5.627010,380.0,-740.0,33.928571
9,Sumner County,196281.0,11438.0,5.827360,6453.0,-4985.0,56.417206


In [439]:
data = pd.read_csv('censusdata.csv')

In [440]:
data.head()

,NAME,Under 6 years,Living with two parents,Living with two parents: Both parents in labor force,Living with two parents: Father only in labor force,Living with two parents: Mother only in labor force,Living with two parents: Neither parent in labor force,Living with one parent,Living with one parent: Living with father,Living with one parent: Living with father: In labor force,Living with one parent: Living with father: Not in labor force,Living with one parent: Living with mother,Living with one parent: Living with mother: In labor force,Living with one parent: Living with mother: Not in labor force
0,Cannon County,875,673,438,200,0,35,202,29,29,0,173,118,55
1,Cheatham County,2521,1712,1079,453,0,180,809,366,322,44,443,302,141
2,Davidson County,52459,34330,21839,10953,1014,524,18129,3742,3406,336,14387,11126,3261
3,Dickson County,3477,1754,1011,550,126,67,1723,429,409,20,1294,940,354
4,Houston County,403,256,93,158,5,0,147,53,53,0,94,84,10


In [441]:
data = data.set_index('NAME').transpose()
thelist = [data['Cannon County'], data['Cheatham County'], data['Davidson County'], data['Dickson County'], data['Macon County'], data['Maury County'], 
        data['Robertson County'], data['Rutherford County'], data['Smith County'], data['Sumner County'], data['Trousdale County'], data['Williamson County'], 
        data['Wilson County'], data['Montgomery County'], data['Stewart County'], data['Houston County'], data['Humphreys County']
       ]
data['Total Region'] = sum(thelist)
thelist = [data['Cannon County'], data['Cheatham County'], data['Davidson County'], data['Dickson County'], data['Macon County'], data['Maury County'], 
        data['Robertson County'], data['Rutherford County'], data['Smith County'], data['Sumner County'], data['Trousdale County'], data['Williamson County'], 
        data['Wilson County']
       ]
data['Nashville MSA'] = sum(thelist)

In [442]:
data = data.transpose().reset_index(drop = False)
data.head()

,NAME,Under 6 years,Living with two parents,Living with two parents: Both parents in labor force,Living with two parents: Father only in labor force,Living with two parents: Mother only in labor force,Living with two parents: Neither parent in labor force,Living with one parent,Living with one parent: Living with father,Living with one parent: Living with father: In labor force,Living with one parent: Living with father: Not in labor force,Living with one parent: Living with mother,Living with one parent: Living with mother: In labor force,Living with one parent: Living with mother: Not in labor force
0,Cannon County,875,673,438,200,0,35,202,29,29,0,173,118,55
1,Cheatham County,2521,1712,1079,453,0,180,809,366,322,44,443,302,141
2,Davidson County,52459,34330,21839,10953,1014,524,18129,3742,3406,336,14387,11126,3261
3,Dickson County,3477,1754,1011,550,126,67,1723,429,409,20,1294,940,354
4,Houston County,403,256,93,158,5,0,147,53,53,0,94,84,10


In [443]:
data.set_index('NAME', inplace = True)
cols = data.columns
data[cols] = data[cols].astype(float)
data = data.reset_index(drop = False)

In [444]:
thelist = [data['               Living with two parents: Both parents in labor force'], 
           data['        Living with one parent: Living with father: In labor force'], 
           data['            Living with one parent: Living with mother: In labor force']
          ]
data['Under 6 with Primary Caregiver Definitely in Labor Force'] = sum(thelist)

In [445]:
data['Under 6 % with Primary Caregiver Definitely in Labor Force'] = percent( data['Under 6 with Primary Caregiver Definitely in Labor Force'], data['    Under 6 years'])

In [446]:
data.head()

,NAME,Under 6 years,Living with two parents,Living with two parents: Both parents in labor force,Living with two parents: Father only in labor force,Living with two parents: Mother only in labor force,Living with two parents: Neither parent in labor force,Living with one parent,Living with one parent: Living with father,Living with one parent: Living with father: In labor force,Living with one parent: Living with father: Not in labor force,Living with one parent: Living with mother,Living with one parent: Living with mother: In labor force,Living with one parent: Living with mother: Not in labor force,Under 6 with Primary Caregiver Definitely in Labor Force,Under 6 % with Primary Caregiver Definitely in Labor Force
0,Cannon County,875.0,673.0,438.0,200.0,0.0,35.0,202.0,29.0,29.0,0.0,173.0,118.0,55.0,585.0,66.857143
1,Cheatham County,2521.0,1712.0,1079.0,453.0,0.0,180.0,809.0,366.0,322.0,44.0,443.0,302.0,141.0,1703.0,67.552559
2,Davidson County,52459.0,34330.0,21839.0,10953.0,1014.0,524.0,18129.0,3742.0,3406.0,336.0,14387.0,11126.0,3261.0,36371.0,69.332240
3,Dickson County,3477.0,1754.0,1011.0,550.0,126.0,67.0,1723.0,429.0,409.0,20.0,1294.0,940.0,354.0,2360.0,67.874605
4,Houston County,403.0,256.0,93.0,158.0,5.0,0.0,147.0,53.0,53.0,0.0,94.0,84.0,10.0,230.0,57.071960


In [447]:
carry = data[['NAME', 'Under 6 with Primary Caregiver Definitely in Labor Force', 'Under 6 % with Primary Caregiver Definitely in Labor Force']]

In [448]:
data = df.merge(carry, on = "NAME")

In [449]:
data.head()

,NAME,Population,Children Under 5,Percent of Population Children Under 5,Licensed Capacity,Difference,Percent of Children Under 5 with a Spot,Under 6 with Primary Caregiver Definitely in Labor Force,Under 6 % with Primary Caregiver Definitely in Labor Force
0,Cannon County,14506.0,789.0,5.439129,164.0,-625.0,20.785805,585.0,66.857143
1,Cheatham County,41072.0,2250.0,5.478185,1881.0,-369.0,83.600000,1703.0,67.552559
2,Davidson County,715884.0,46175.0,6.450067,28570.0,-17605.0,61.873308,36371.0,69.332240
3,Dickson County,54315.0,3168.0,5.832643,1328.0,-1840.0,41.919192,2360.0,67.874605
4,Macon County,25216.0,1736.0,6.884518,498.0,-1238.0,28.686636,1576.0,76.208897


In [450]:
data['Children Under 5 with Both Parents or Single Parent in Labor Force'] = data['Children Under 5'] * (data['Under 6 % with Primary Caregiver Definitely in Labor Force']/100)

In [451]:
data.head()

,NAME,Population,Children Under 5,Percent of Population Children Under 5,Licensed Capacity,Difference,Percent of Children Under 5 with a Spot,Under 6 with Primary Caregiver Definitely in Labor Force,Under 6 % with Primary Caregiver Definitely in Labor Force,Children Under 5 with Both Parents or Single Parent in Labor Force
0,Cannon County,14506.0,789.0,5.439129,164.0,-625.0,20.785805,585.0,66.857143,527.502857
1,Cheatham County,41072.0,2250.0,5.478185,1881.0,-369.0,83.600000,1703.0,67.552559,1519.932566
2,Davidson County,715884.0,46175.0,6.450067,28570.0,-17605.0,61.873308,36371.0,69.332240,32014.162012
3,Dickson County,54315.0,3168.0,5.832643,1328.0,-1840.0,41.919192,2360.0,67.874605,2150.267472
4,Macon County,25216.0,1736.0,6.884518,498.0,-1238.0,28.686636,1576.0,76.208897,1322.986460


In [452]:
data['Percent of Children Under 5 with Both Parents or Single Parent in Labor force'] = percent( data['Children Under 5 with Both Parents or Single Parent in Labor Force'], 
                                                                                                data['Children Under 5'])
data['Difference Between Children With Parent(s) in Labor Force and Capacity'] = data['Licensed Capacity'] - data['Children Under 5 with Both Parents or Single Parent in Labor Force']
data['Children With Parent(s) in Labor Force: % With a Spot'] = percent(data['Licensed Capacity'], data['Children Under 5 with Both Parents or Single Parent in Labor Force'] 
                                                                       )

In [453]:
data = data[['NAME', 'Children Under 5', 
             'Children Under 5 with Both Parents or Single Parent in Labor Force', 
             'Percent of Children Under 5 with Both Parents or Single Parent in Labor force', 
             'Licensed Capacity', 'Difference', 'Difference Between Children With Parent(s) in Labor Force and Capacity', 
             'Children With Parent(s) in Labor Force: % With a Spot'
             
            ]]

In [454]:
data

,NAME,Children Under 5,Children Under 5 with Both Parents or Single Parent in Labor Force,Percent of Children Under 5 with Both Parents or Single Parent in Labor force,Licensed Capacity,Difference,Difference Between Children With Parent(s) in Labor Force and Capacity,Children With Parent(s) in Labor Force: % With a Spot
0,Cannon County,789.0,527.502857,66.857143,164.0,-625.0,-363.502857,31.089879
1,Cheatham County,2250.0,1519.932566,67.552559,1881.0,-369.0,361.067434,123.755490
2,Davidson County,46175.0,32014.162012,69.332240,28570.0,-17605.0,-3444.162012,89.241755
3,Dickson County,3168.0,2150.267472,67.874605,1328.0,-1840.0,-822.267472,61.759759
4,Macon County,1736.0,1322.986460,76.208897,498.0,-1238.0,-824.986460,37.642109
5,Maury County,6532.0,4581.721186,70.142700,2724.0,-3808.0,-1857.721186,59.453640
6,Robertson County,4456.0,2583.947581,57.988052,2237.0,-2219.0,-346.947581,86.572964
7,Rutherford County,21410.0,13754.662730,64.244104,12710.0,-8700.0,-1044.662730,92.405028
8,Smith County,1120.0,740.560748,66.121495,380.0,-740.0,-360.560748,51.312468
9,Sumner County,11438.0,7365.094436,64.391453,6453.0,-4985.0,-912.094436,87.615985


In [455]:
data['Children With Parent(s) in Labor Force: % Without a Spot'] = 100 - data['Children With Parent(s) in Labor Force: % With a Spot']

In [456]:
data.to_csv('final.csv', index = False)